In [2]:
import numpy as np
from numpy import nan

from tabulate import tabulate
from itertools import zip_longest 

In [3]:
class Stack():

  def __init__(self, size):

      if size > 0 :
        self.stack = np.arange(size)
        self.top = self.stack[-1]
      else :
        self.stack = np.array([])
        self.top = None


In [4]:
class Moove():

  def __init__(self, A, B):
      self.A = A
      self.B = B

  
  def push_a(self):
        # Si A est vide 
        if not self.A.stack.size:
          return
        self.B.stack = np.append(self.B.stack, self.A.top)
        self.A.stack = np.delete(self.A.stack, -1)
        self.B.top = self.A.top

        # Si A est vide maintenant
        if not self.A.stack.size:
          self.A.top = None
        else:
          self.A.top = self.A.stack[-1]
    
  def push_b(self):
      # Si B est vide 
      if not self.B.stack.size:
        return
      # Si A est vide
      if not self.A.stack.size :
        self.A.stack = np.append(self.A.stack, self.B.top)
        self.B.stack = np.delete(self.B.stack, -1)
        self.A.top = self.B.top

      # Si B est vide maintenant
      if not self.B.stack.size:
        self.B.top = None
      else:
        self.B.top = self.B.stack[-1]
  
  def rotate_a(self):
    
    # Si A est vide ou n'a qu'un element 
    if not self.A.stack.size or self.A.stack.size == 1:
      return

    self.A.stack = np.roll(self.A.stack, 1)
    self.A.top = self.A.stack[-1]

  def rotate_b(self):
    
    # Si B est vide ou n'a qu'un element 
    if not self.B.stack.size or self.B.stack.size == 1:
      return
    
    self.B.stack = np.roll(self.B.stack, 1)
    self.B.top = self.B.stack[-1]

  def rotate(self):
    
    self.rotate_a()
    self.rotate_b()

  def inverse_rotate_a(self):

    # Si A est vide ou n'a qu'un element 
    if not self.A.stack.size or self.A.stack.size == 1:
      return
    
    self.A.stack = np.roll(self.A.stack, -1)
    self.A.top = self.A.stack[-1]
  
  def inverse_rotate_b(self):

    # Si B est vide ou n'a qu'un element 
    if not self.B.stack.size or self.B.stack.size == 1:
      return

    self.B.stack = np.roll(self.B.stack, -1)
    self.B.top = self.B.stack[-1]

  def reverse(self):
    
    self.inverse_rotate_a()
    self.inverse_rotate_b()
  
  def swap_a(self):
      # Si A est vide ou n'a qu'un element 
      if not self.A.stack.size or self.A.stack.size == 1:
        return

      self.A.stack[[-1, -2]] =  self.A.stack[[-2, -1]]
      self.A.top = self.A.stack[-1]

  def swap_b(self):
    
      # Si B est vide ou n'a qu'un element 
      if not self.B.stack.size or self.B.stack.size == 1:
        return
    
      self.B.stack[[-1, -2]] =  self.B.stack[[-2, -1]]
      self.B.top = self.B.stack[-1]
      
  def swap(self):
    
    self.swap_a()
    self.swap_b()
    
    
    
    

In [8]:
class Action():
    
    def __init__(self):
        
        self.push_a = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        self.push_b = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        self.rotate_a = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
        self.rotate_b = np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
        self.rotate = np.array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])
        self.inverse_rotate_a = np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
        self.inverse_rotate_b = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])
        self.reverse = np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
        self.swap_a = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
        self.swap_b = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
        self.swap = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
        
        self.possible_action = np.array([self.push_a, 
                                         self.push_b, 
                                         self.rotate_a, 
                                         self.rotate_b,
                                         self.rotate,
                                         self.inverse_rotate_a,
                                         self.inverse_rotate_b,
                                         self.reverse,
                                         self.swap_a,
                                         self.swap_b,
                                         self.swap])
        
        self.actions_space = len(self.possible_action)
    

In [13]:
test = Action()
possible = test.possible_action
possible

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [5]:
class Env():
    
    def __init__(self, size):
        self.A = Stack(size)
        self.B = Stack(0)
        self.cum_reward = 0
        self.moover = Moove(self.A, self.B)

        
    def print_stack(self):
        print("-------------------")
        for a, b in zip_longest(reversed(self.A.stack), reversed(self.B.stack), fillvalue='xxx'):
          
          print(f"{a}\t{b}")
        print("-------------------")
  


In [22]:
env = Env(10)

In [23]:
env.print_stack()
env.moover.swap_a()
env.moover.push_a()
env.moover.swap_b()
env.print_stack()

-------------------
9	xxx
8	xxx
7	xxx
6	xxx
5	xxx
4	xxx
3	xxx
2	xxx
1	xxx
0	xxx
-------------------
-------------------
9	8.0
7	xxx
6	xxx
5	xxx
4	xxx
3	xxx
2	xxx
1	xxx
0	xxx
-------------------
